In [1]:
from google.colab import drive
drive.mount("/content/gdrive")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


# Precprocessing

In [0]:
fp = open("/content/gdrive/Shared drives/NLP/Assignment 3/train.txt", 'r')
raw_train=[]
q=0
while(q!=15131):
  lst=[]
  q=q+1
  for line in fp:
    line = line.strip()
    if len(line.split())>1:
      lst.append(line.split())
    if line=='':
      raw_train.append(lst)
      break

In [0]:
raw_test[:2]

In [0]:
fp = open("/content/gdrive/Shared drives/NLP/Assignment 3/test.txt", 'r')
raw_test=[]
q=0
while(q!=1869):
  lst=[]
  q=q+1
  for line in fp:
    line = line.strip()
    if len(line.split())>1:
      lst.append(line.split())
    if line=='':
      raw_test.append(lst)
      break

In [0]:
y_train = []
train_id = []
for tweet in raw_train:
  train_id.append(tweet[0][1])
  y_train.append(tweet[0][2])

y_test = []
test_id = []
for tweet in raw_test:
  test_id.append(tweet[0][1])
  y_test.append(tweet[0][2])

In [56]:
from collections import Counter
train_dist = Counter(y_train)
train_dist

Counter({'negative': 4459, 'neutral': 5638, 'positive': 5034})

In [57]:
#from collections import Counter
test_dist = Counter(y_test)
test_dist

Counter({'negative': 533, 'neutral': 754, 'positive': 582})

In [0]:
# with open("y_train.txt",'w') as f:
#   for id1, sentiment in zip(train_id, y_train):
#       f.write(id1+"\t"+sentiment+"\n")
  
# with open("y_test.txt",'w') as f:
#   for id1, sentiment in zip(test_id, y_test):
#       f.write(id1+"\t"+sentiment+"\n")

Pre-processing is done using regex, since we already have words tokenized, if token matches [a-zA-Z], it is accepted else it is rejected.

But URL token such as 'http', 't', and 'co', is not filtered from above regex so we use conditional regex to filter it

In [0]:
import re
new_train = []
for i in range(15131):
  temp = []
  for j in range(len(raw_train[i])):
    if bool(re.match("^[a-zA-Z]+$",raw_train[i][j][0])):
      if bool(re.match("^(http)|(^t$)|(^co$)",raw_train[i][j][0])):
        continue
      else:
        raw_train[i][j][0] = raw_train[i][j][0].lower()
        temp.append(raw_train[i][j])
  new_train.append(temp[1:])

X_train = []
for tweet in new_train:
  X_train.append(" ".join([j[0] for j in tweet]))

In [0]:
X_train[:2]

In [0]:
import re
new_test = []
for i in range(1869):
  temp = []
  for j in range(len(raw_test[i])):
    if bool(re.match("^[a-zA-Z]+$",raw_test[i][j][0])):
      if bool(re.match("^(http)|(^t$)|(^co$)",raw_test[i][j][0])):
        continue
      else:
        raw_test[i][j][0] = raw_test[i][j][0].lower()
        temp.append(raw_test[i][j])
  new_test.append(temp[1:])

X_test = []
for tweet in new_test:
  X_test.append(" ".join([j[0] for j in tweet]))

In [0]:
# with open("X_train.txt",'w') as f:
#   for id1, tweet in zip(train_id, final_new_train):
#       f.write(id1+"\t"+" ".join([j[0] for j in tweet[1:-1]])+"\n")

# with open("X_test.txt",'w') as f:
#   for id1, tweet in zip(test_id, final_new_test):
#       f.write(id1+"\t"+" ".join([j[0] for j in tweet[1:-1]])+"\n")

# CSNLI

In [0]:
fp = open("/content/gdrive/Shared drives/NLP/Assignment 3/train_tweets_translated.txt", 'r')

X_train_csnli=[]

tweets = []
for line in fp:
  if line != "\n":
    tweets.append(line[:-1])

raw = []
csnli = []
english = []


i = 0
while i < len(tweets)-2:
  raw.append(tweets[i])
  csnli.append(tweets[i+1])
  english.append(tweets[i+2])
  i+=3

In [88]:
for i in range(5):
  print(raw[i])
  print(csnli[i])
  print(english[i])
  print("\n")

pakistan ka ghra tauq he pakistan israel ko tasleem nahein kerta isko palestine kehta he occupied
pakistan का गहरा तौक़ है pakistan israel को तस्लीम नहीं करता इसको palestine कहते हैं occupied
pakistan has a deep stigma pakistan does not destroy israel, it is called palestine occupied


mulle ye mathura me nahi dikha tha jab mullo ne hindu ko iss liye mara ki vo lasse ki paise mag liye
मुल्ले यह mathura में नहीं दिखा था जब मुल्लों ने हिन्दू को इस लिए मारा कि वह लस्सी की पैसे मग लिए
Mullahs did not show this in mathura when the mullahs hit the Hindu for mug of lassi


manya pradhan mantri mahoday shriman narendra modi ji pradhanmantri banne par hardik badhai tahe
मान्य pradhan mantri महोदय shriman narendra modi जी प्रधानमंत्री बनने पर हार्दिक बधाई ताहि
Valid pradhan mantri sir shriman narendra modi ji hearty congratulations on becoming prime minister


krishna jcb full trend me chal rahi
krishna jcb full trend में चल रही
krishna jcb operates in full trend


ravishkumarblog loksabha me ja

# BERT Embeddings

In [0]:
!pip install flair

In [10]:
from flair.data import Sentence

X_train_flair = [Sentence(tweet) for tweet in X_train]

In [0]:
from flair.data import Sentence

X_test_flair = [Sentence(tweet) for tweet in X_test]

In [0]:
from flair.embeddings import BertEmbeddings
import numpy as np
embedding_bert = BertEmbeddings("bert-base-cased")

X_train_bert = []
for sent in X_train_flair:
  sent_emb = np.array(embedding_bert.embed(sent)[0])
  emb = []
  for token in sent_emb:
    emb.append(np.array(token.embedding.cpu()))
  X_train_bert.append(np.array(emb))

In [15]:
EMBEDDINGS_SIZE = X_train_bert[0].shape[1]
EMBEDDINGS_SIZE

3072

In [0]:
X_test_bert = []
for sent in X_test_flair:
  sent_emb = np.array(embedding_bert.embed(sent)[0])
  emb = []
  for token in sent_emb:
    emb.append(np.array(token.embedding.cpu()))
  X_test_bert.append(np.array(emb))

In [13]:
MAX_SEQ_LEN_train = max([len(tweet.split()) for tweet in X_train])
print(MAX_SEQ_LEN_train)

MAX_SEQ_LEN_test = max([len(tweet.split()) for tweet in X_test])
print(MAX_SEQ_LEN_test)

MAX_SEQ_LEN = min(MAX_SEQ_LEN_train, MAX_SEQ_LEN_test)
MAX_SEQ_LEN

35
31


31

In [0]:
import numpy as np
X_train_bert = np.load("/content/gdrive/Shared drives/NLP/Assignment 3/bert_train.npy", allow_pickle=True)

In [0]:
import numpy as np
X_test_bert = np.load("/content/gdrive/Shared drives/NLP/Assignment 3/bert_test.npy", allow_pickle=True)

In [0]:
import torch
from torch.nn.utils.rnn import pad_sequence

emb_list = []
for i in X_train_bert:
  emb_list.append(torch.tensor(i[:31]))
X_train_bert_padded = pad_sequence(emb_list, batch_first=True)

emb_list = []
for i in X_test_bert:
  emb_list.append(torch.tensor(i[:MAX_SEQ_LEN]))
X_test_bert_padded = pad_sequence(emb_list, batch_first=True)

In [67]:
y_dct = {'negative':0, "neutral":1,"positive":2}
print(y_train[:5])

for i, sentiment in enumerate(y_train):
  y_train[i] = y_dct[sentiment]

import numpy as np 
y_train = np.array(y_train)
print(y_train[:5])

from sklearn.preprocessing import OneHotEncoder

enc = OneHotEncoder(handle_unknown='ignore')
enc.fit(y_train.reshape(-1, 1))
y_train = enc.transform(y_train.reshape(-1, 1)).toarray()

print(y_train[:5])

['negative', 'negative', 'positive', 'positive', 'positive']
[0 0 2 2 2]
[[1. 0. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]]


In [68]:
y_dct = {'negative':0, "neutral":1,"positive":2}
print(y_test[:5])

y_test_int = []
for i, sentiment in enumerate(y_test):
  y_test_int.append(y_dct[sentiment])

import numpy as np 
y_test_int = np.array(y_test_int)
print(y_test_int[:5])

from sklearn.preprocessing import OneHotEncoder

enc = OneHotEncoder(handle_unknown='ignore')
enc.fit(y_test_int.reshape(-1, 1))
y_test= enc.transform(y_test_int.reshape(-1, 1)).toarray()

print(y_test[:5])

['neutral', 'neutral', 'neutral', 'negative', 'positive']
[1 1 1 0 2]
[[0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 0. 1.]]


In [69]:
!pip install keras_Self_attention
!pip install keras_tqdm

In [0]:
import pickle

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import GRU
from keras.layers import Bidirectional
from keras import optimizers
from keras.layers.recurrent import LSTM
import keras
from keras.layers import Dropout
from keras_self_attention import SeqSelfAttention
from keras_tqdm import TQDMNotebookCallback

In [19]:
#------------------------------------MODELS------------------------------------------#
 
"""#### Basic BiLSTM, no appends
**MODEL 0.0**:
*   BERT
*   2 LSTM layer
*   4 dense
*   2 dense
"""

model_Bi_LSTM_1 = Sequential()
model_Bi_LSTM_1.add(Bidirectional(LSTM(10, dropout=0.5, recurrent_dropout=0.5, return_sequences=True), 
                                  input_shape=(31, 3072), 
                                  merge_mode='concat'))
model_Bi_LSTM_1.add(Bidirectional(LSTM(10, dropout=0.5, recurrent_dropout=0.5), merge_mode='concat'))
model_Bi_LSTM_1.add(Dense(5, activation='softmax'))
model_Bi_LSTM_1.add(Dense(3, activation='softmax'))
model_Bi_LSTM_1.summary()
model_Bi_LSTM_1.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])
model_Bi_LSTM_1.fit(x=X_train_bert_padded.cpu().numpy(), 
                    y=y_train, 
                    validation_data=(X_test_bert_padded.cpu().numpy(), y_test),	
                    batch_size=31, 
                    epochs=25, 
                    shuffle=True)






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_1 (Bidirection (None, 31, 20)            246640    
_________________________________________________________________
bidirectional_2 (Bidirection (None, 20)                2480      
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 105       
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 18        
Total params: 249,243
Trainable params: 249,243
Non-trainable params: 0
_________________________________________________________________


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Train on 15131 samples, vali

In [22]:
model_Bi_LSTM_1.evaluate(X_test_bert_padded.cpu().numpy(), y_test)[1]

1869/1869 [==============================] - 3s 2ms/step


0.5468164795602051

In [0]:
y_pred = []          

for tweet in X_test_bert_padded.cpu().numpy():
  y_pred.append(model_Bi_LSTM_1.predict_classes(np.array([tweet])))

y_pred = np.array([i[0] for i in y_pred])

In [40]:
from sklearn.metrics import classification_report as clf
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import pandas as pd

target_names=['negative','neutral','positive']

df_cm = pd.DataFrame(confusion_matrix(y_test_int, y_pred), columns=target_names, index=target_names)
df_cm.index.name = 'Actual'
df_cm.columns.name = 'Predicted'
print(df_cm)
print("\n")
print("Accuracy: ",accuracy_score(y_test_int, y_pred))
print("\n","Report:")
print(clf(y_test_int, y_pred, target_names=target_names))

Predicted  negative  neutral  positive
Actual                                
negative        352      133        48
neutral         235      333       186
positive        100      145       337


Accuracy:  0.5468164794007491

 Report:
              precision    recall  f1-score   support

    negative       0.51      0.66      0.58       533
     neutral       0.55      0.44      0.49       754
    positive       0.59      0.58      0.58       582

    accuracy                           0.55      1869
   macro avg       0.55      0.56      0.55      1869
weighted avg       0.55      0.55      0.54      1869



In [71]:
"""
#### Self Attention Library, no appends
**MODEL 1.1 SelfAtt**
"""

model_Bi_LSTM_att1 = Sequential()
model_Bi_LSTM_att1.add(Bidirectional(LSTM(10, dropout=0.5, recurrent_dropout=0.5, return_sequences=True), input_shape=(31, 3072), merge_mode='concat'))
model_Bi_LSTM_att1.add(SeqSelfAttention(attention_activation='sigmoid'))
model_Bi_LSTM_att1.add(Bidirectional(LSTM(5, dropout=0.5, recurrent_dropout=0.5), merge_mode='concat'))
#model_Bi_LSTM_att1.add(Dense(5, activation='softmax'))
model_Bi_LSTM_att1.add(Dense(3, activation='softmax'))
model_Bi_LSTM_att1.summary()
model_Bi_LSTM_att1.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])
model_Bi_LSTM_att1.fit(x=X_train_bert_padded.cpu().numpy(), 
                       y=y_train, 
                       validation_data=(X_test_bert_padded.cpu().numpy(), y_test),	
                       batch_size=31, 
                       epochs=15, 
                       shuffle=True)


Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_12 (Bidirectio (None, 31, 20)            246640    
_________________________________________________________________
seq_self_attention_6 (SeqSel (None, 31, 20)            1345      
_________________________________________________________________
bidirectional_13 (Bidirectio (None, 10)                1040      
_________________________________________________________________
dense_8 (Dense)              (None, 3)                 33        
Total params: 249,058
Trainable params: 249,058
Non-trainable params: 0
_________________________________________________________________
Train on 15131 samples, validate on 1869 samples
Epoch 1/15
15131/15131 [==============================] - 94s 6ms/step - loss: 0.9922 - acc: 0.4797 - val_loss: 0.9991 - val_acc: 0.5078
Epoch 2/15
15131/15131 [==============================] - 84

In [72]:
model_Bi_LSTM_att1.evaluate(X_test_bert_padded.cpu().numpy(), y_test)[1]

1869/1869 [==============================] - 5s 2ms/step


0.5510968433273483

In [0]:
y_pred = []          

for tweet in X_test_bert_padded.cpu().numpy():
  y_pred.append(model_Bi_LSTM_att1.predict_classes(np.array([tweet])))

y_pred = np.array([i[0] for i in y_pred])

In [74]:
from sklearn.metrics import classification_report as clf
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import pandas as pd

target_names=['negative','neutral','positive']

df_cm = pd.DataFrame(confusion_matrix(y_test_int, y_pred), columns=target_names, index=target_names)
df_cm.index.name = 'Actual'
df_cm.columns.name = 'Predicted'
print(df_cm)
print("\n")
print("Accuracy: ",accuracy_score(y_test_int, y_pred))
print("\n","Report:")
print(clf(y_test_int, y_pred, target_names=target_names))

Predicted  negative  neutral  positive
Actual                                
negative        320      169        44
neutral         209      373       172
positive         91      154       337


Accuracy:  0.5510968432316747

 Report:
              precision    recall  f1-score   support

    negative       0.52      0.60      0.56       533
     neutral       0.54      0.49      0.51       754
    positive       0.61      0.58      0.59       582

    accuracy                           0.55      1869
   macro avg       0.55      0.56      0.55      1869
weighted avg       0.55      0.55      0.55      1869



In [75]:
from keras.layers import GRU
model_GRU = Sequential()
model_GRU.add(Bidirectional(LSTM(10, dropout=0.5, recurrent_dropout=0.5, return_sequences=True), input_shape=(31, 3072), merge_mode='concat'))
#model_GRU.add(GRU(8, dropout=0.5, recurrent_dropout=0.5, return_sequences=True))
model_GRU.add(SeqSelfAttention(attention_activation='sigmoid'))
model_GRU.add(GRU(8, dropout=0.5, recurrent_dropout=0.5, return_sequences=False))
#model_GRU.add(Dense(8, activation='softmax'))
model_GRU.add(Dense(3, activation='softmax'))
model_GRU.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])
model_GRU.fit(x=X_train_bert_padded.cpu().numpy(), 
              y=y_train, 
              validation_data=(X_test_bert_padded.cpu().numpy(), y_test),	
              batch_size=31, epochs=15, shuffle=True)


Train on 15131 samples, validate on 1869 samples
Epoch 1/15
15131/15131 [==============================] - 84s 6ms/step - loss: 0.9971 - acc: 0.4814 - val_loss: 0.9962 - val_acc: 0.5040
Epoch 2/15
15131/15131 [==============================] - 76s 5ms/step - loss: 0.9262 - acc: 0.5390 - val_loss: 0.9838 - val_acc: 0.5313
Epoch 3/15
15131/15131 [==============================] - 77s 5ms/step - loss: 0.9070 - acc: 0.5510 - val_loss: 0.9815 - val_acc: 0.5163
Epoch 4/15
15131/15131 [==============================] - 75s 5ms/step - loss: 0.8928 - acc: 0.5688 - val_loss: 0.9800 - val_acc: 0.5377
Epoch 5/15
15131/15131 [==============================] - 80s 5ms/step - loss: 0.8771 - acc: 0.5797 - val_loss: 0.9797 - val_acc: 0.5361
Epoch 6/15
15131/15131 [==============================] - 75s 5ms/step - loss: 0.8739 - acc: 0.5732 - val_loss: 0.9802 - val_acc: 0.5324
Epoch 7/15
15131/15131 [==============================] - 77s 5ms/step - loss: 0.8677 - acc: 0.5824 - val_loss: 0.9757 - val_acc:

In [81]:
model_GRU.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_14 (Bidirectio (None, 31, 20)            246640    
_________________________________________________________________
seq_self_attention_7 (SeqSel (None, 31, 20)            1345      
_________________________________________________________________
gru_2 (GRU)                  (None, 8)                 696       
_________________________________________________________________
dense_9 (Dense)              (None, 3)                 27        
Total params: 248,708
Trainable params: 248,708
Non-trainable params: 0
_________________________________________________________________


In [78]:
model_GRU.evaluate(X_test_bert_padded.cpu().numpy(), y_test)[1]

1869/1869 [==============================] - 4s 2ms/step


0.5420010702823049

In [0]:
y_pred = []          

for tweet in X_test_bert_padded.cpu().numpy():
  y_pred.append(model_GRU.predict_classes(np.array([tweet])))

y_pred = np.array([i[0] for i in y_pred])

In [80]:
from sklearn.metrics import classification_report as clf
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import pandas as pd

target_names=['negative','neutral','positive']

df_cm = pd.DataFrame(confusion_matrix(y_test_int, y_pred), columns=target_names, index=target_names)
df_cm.index.name = 'Actual'
df_cm.columns.name = 'Predicted'
print(df_cm)
print("\n")
print("Accuracy: ",accuracy_score(y_test_int, y_pred))
print("\n","Report:")
print(clf(y_test_int, y_pred, target_names=target_names))

Predicted  negative  neutral  positive
Actual                                
negative        299      184        50
neutral         195      380       179
positive         79      169       334


Accuracy:  0.5420010700909578

 Report:
              precision    recall  f1-score   support

    negative       0.52      0.56      0.54       533
     neutral       0.52      0.50      0.51       754
    positive       0.59      0.57      0.58       582

    accuracy                           0.54      1869
   macro avg       0.54      0.55      0.55      1869
weighted avg       0.54      0.54      0.54      1869



# BERT sequence classification

In [0]:
import tensorflow as tf

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [0]:
!pip install pytorch-pretrained-bert pytorch-nlp

     |████████████████████████████████| 133kB 2.7MB/s 
     |████████████████████████████████| 92kB 11.1MB/s 
     |████████████████████████████████| 645kB 48.6MB/s 


In [0]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from pytorch_pretrained_bert import BertTokenizer, BertConfig
from pytorch_pretrained_bert import BertAdam, BertForSequenceClassification
from tqdm import tqdm, trange
import pandas as pd
import io
import numpy as np
import matplotlib.pyplot as plt
% matplotlib inline

Using TensorFlow backend.


In [0]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)

'Tesla K80'

In [0]:
# Create sentence and label lists
sentences = X_train

# We need to add special tokens at the beginning and end of each sentence for BERT to work properly
sentences = ["[CLS] " + sentence + " [SEP]" for sentence in sentences]
labels = y_train

In [0]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]


In [0]:
for tweet, bert_tokens, input_id in zip(X_train[:5],tokenized_texts[:5],input_ids[:5]):
  print(tweet)
  print(bert_tokens)
  print(input_id)
  print("\n")

pakistan ka ghra tauq he pakistan israel ko tasleem nahein kerta isko palestine kehta he occupied
['[CLS]', 'pakistan', 'ka', 'g', '##hra', 'tau', '##q', 'he', 'pakistan', 'israel', 'ko', 'ta', '##sle', '##em', 'nah', '##ein', 'ke', '##rta', 'is', '##ko', 'palestine', 'ke', '##hta', 'he', 'occupied', '[SEP]']
[101, 4501, 10556, 1043, 13492, 19982, 4160, 2002, 4501, 3956, 12849, 11937, 25016, 6633, 20976, 12377, 17710, 13320, 2003, 3683, 8976, 17710, 22893, 2002, 4548, 102]


mulle ye mathura me nahi dikha tha jab mullo ne hindu ko iss liye mara ki vo lasse ki paise mag liye
['[CLS]', 'mu', '##lle', 'ye', 'math', '##ura', 'me', 'nah', '##i', 'di', '##kha', 'tha', 'ja', '##b', 'mu', '##llo', 'ne', 'hindu', 'ko', 'iss', 'li', '##ye', 'mara', 'ki', 'vo', 'lass', '##e', 'ki', 'pa', '##ise', 'mag', 'li', '##ye', '[SEP]']
[101, 14163, 6216, 6300, 8785, 4648, 2033, 20976, 2072, 4487, 15256, 22794, 14855, 2497, 14163, 7174, 11265, 7560, 12849, 26354, 5622, 6672, 13955, 11382, 29536, 27333, 2063

In [0]:
MAX_LEN = max([len(bert_tokens) for bert_tokens in tokenized_texts])
print(MAX_LEN)

55


In [0]:
# Pad our input tokens
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

In [0]:
# Create attention masks
attention_masks = []

# Create a mask of 1s for each token followed by 0s for padding
for seq in input_ids:
  seq_mask = [float(i>0) for i in seq]
  attention_masks.append(seq_mask)

In [0]:
# Use train_test_split to split our data into train and validation sets for training

train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids, labels, 
                                                            random_state=2018, test_size=0.1)
train_masks, validation_masks, _, _ = train_test_split(attention_masks, input_ids,
                                             random_state=2018, test_size=0.1)

In [0]:
# Convert all of our data into torch tensors, the required datatype for our model

train_inputs = torch.tensor(train_inputs)
validation_inputs = torch.tensor(validation_inputs)
train_labels = torch.tensor(train_labels)
validation_labels = torch.tensor(validation_labels)
train_masks = torch.tensor(train_masks)
validation_masks = torch.tensor(validation_masks)

In [0]:
# Select a batch size for training. For fine-tuning BERT on a specific task, the authors recommend a batch size of 16 or 32
batch_size = 16

# Create an iterator of our data with torch DataLoader. This helps save on memory during training because, unlike a for loop, 
# with an iterator the entire dataset does not need to be loaded into memory

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

In [0]:
# Load BertForSequenceClassification, the pretrained BERT model with a single linear classification layer on top. 

model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=3)
model.cuda()

100%|██████████| 407873900/407873900 [00:32<00:00, 12509784.49B/s]


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): BertLayerNorm()
              (dropout): Dropout(p=0.1, inplace=False)
   

In [0]:
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.0}
]

In [0]:
# This variable contains all of the hyperparemeter information our training loop needs
optimizer = BertAdam(optimizer_grouped_parameters,
                     lr=2e-5,
                     warmup=.1)

t_total value of -1 results in schedule not being applied


In [0]:
# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [0]:
# Store our loss and accuracy for plotting
train_loss_set = []

# Number of training epochs (authors recommend between 2 and 4)
epochs = 10

# trange is a tqdm wrapper around the normal python range
for _ in trange(epochs, desc="Epoch"):
  
  
  # Training
  
  # Set our model to training mode (as opposed to evaluation mode)
  model.train()
  
  # Tracking variables
  tr_loss = 0
  nb_tr_examples, nb_tr_steps = 0, 0
  
  # Train the data for one epoch
  for step, batch in enumerate(train_dataloader):
    # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)
    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask, b_labels = batch
    # Clear out the gradients (by default they accumulate)
    optimizer.zero_grad()
    # Forward pass
    loss = model(b_input_ids, attention_mask=b_input_mask, labels=b_labels)
    train_loss_set.append(loss.item())    
    # Backward pass
    loss.backward()
    # Update parameters and take a step using the computed gradient
    optimizer.step()
    
    
    # Update tracking variables
    tr_loss += loss.item()
    nb_tr_examples += b_input_ids.size(0)
    nb_tr_steps += 1

  print("Train loss: {}".format(tr_loss/nb_tr_steps))
    
    
  # Validation

  # Put model in evaluation mode to evaluate loss on the validation set
  model.eval()

  # Tracking variables 
  eval_loss, eval_accuracy = 0, 0
  nb_eval_steps, nb_eval_examples = 0, 0

  # Evaluate data for one epoch
  for batch in validation_dataloader:
    # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)
    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask, b_labels = batch
    # Telling the model not to compute or store gradients, saving memory and speeding up validation
    with torch.no_grad():
      # Forward pass, calculate logit predictions
      logits = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
    
    # Move logits and labels to CPU
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()

    tmp_eval_accuracy = flat_accuracy(logits, label_ids)
    
    eval_accuracy += tmp_eval_accuracy
    nb_eval_steps += 1

  print("Validation Accuracy: {}".format(eval_accuracy/nb_eval_steps))

Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Train loss: 0.90925086908777


Epoch:  10%|█         | 1/10 [07:03<1:03:28, 423.19s/it]

Validation Accuracy: 0.6118421052631579
Train loss: 0.7690364541791974


Epoch:  20%|██        | 2/10 [14:06<56:26, 423.31s/it]  

...

Validation Accuracy: 0.6024999999999999
Train loss: 0.12418516706063154


Epoch:  90%|█████████ | 9/10 [1:03:40<07:04, 424.75s/it]

Validation Accuracy: 0.6005263157894737
Train loss: 0.09802631759675036


Epoch: 100%|██████████| 10/10 [1:10:44<00:00, 424.69s/it]

Validation Accuracy: 0.5851315789473684


In [0]:
torch.save(model,"NLP3BERTSC.bin")

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type BertForSequenceClassification. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type BertModel. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type BertEmbeddings. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Embedding. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be check

In [0]:
model=torch.load('NLP3BERTSC.bin')

In [0]:
def predict(sentence):
    #model=torch.load('Model.bin')
    sentences = "[CLS] " + sentence +" [SEP]"
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
    tokenized_texts = tokenizer.tokenize(sentences)
#     print(tokenized_texts)

    #MAX_LEN = 128

    # Use the BERT tokenizer to convert the tokens to their index numbers in the BERT vocabulary
    my_input_ids = [tokenizer.convert_tokens_to_ids(tokenized_texts)]
    # # Pad our input tokens
    my_input_ids = pad_sequences(my_input_ids, maxlen=MAX_LEN, dtype="long",truncating="post", padding="post")
    # # Create attention masks
    my_attention_masks = []
    # print(my_input_ids)
    # # Create a mask of 1s for each token followed by 0s for padding
#     print(my_input_ids)
    for seq in my_input_ids:
        seq_mask = [float(i>0) for i in seq]
        my_attention_masks.append(seq_mask)

    prediction_inputs = torch.tensor(my_input_ids)
    prediction_masks = torch.tensor(my_attention_masks)


    with torch.no_grad():
        prediction_inputs=prediction_inputs.to(device)
        prediction_masks=prediction_masks.to(device)
        logits = model(prediction_inputs.long(), attention_mask=prediction_masks.long())
        logits = logits.detach().cpu().numpy()
    return np.argmax(logits)

In [0]:
y_pred = []          

for tweet in X_test:
  y_pred.append(predict(tweet)) 

In [0]:
f = open("y_pred_BERTSC.txt","w")

for i in y_pred:
  f.write(str(i))
  f.write("\n")  

In [0]:
from sklearn.metrics import classification_report as clf
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import pandas as pd

target_names=['negative','neutral','positive']

df_cm = pd.DataFrame(confusion_matrix(y_test, y_pred), columns=target_names, index=target_names)
df_cm.index.name = 'Actual'
df_cm.columns.name = 'Predicted'
print(df_cm)
print("\n")
print("Accuracy: ",accuracy_score(y_test, y_pred))
print("\n","Report:")
print(clf(y_test, y_pred, target_names=target_names))

Predicted  negative  neutral  positive
Actual                                
negative        314      142        77
neutral         179      299       276
positive         66      136       380


Accuracy:  0.5313001605136437

 Report:
              precision    recall  f1-score   support

    negative       0.56      0.59      0.58       533
     neutral       0.52      0.40      0.45       754
    positive       0.52      0.65      0.58       582

    accuracy                           0.53      1869
   macro avg       0.53      0.55      0.53      1869
weighted avg       0.53      0.53      0.53      1869



In [0]:
# [[314 142  77]
#  [179 299 276]
#  [ 66 136 380]]
# 0.5313001605136437
#               precision    recall  f1-score   support

#     negative       0.56      0.59      0.58       533
#      neutral       0.52      0.40      0.45       754
#     positive       0.52      0.65      0.58       582

#     accuracy                           0.53      1869
#    macro avg       0.53      0.55      0.53      1869
# weighted avg       0.53      0.53      0.53      1869
